# AI Art Image Classifier Model

## Setup

In [ ]:
!pip install --upgrade huggingface_hub datasets transformers evaluate accelerate pillow torchvision scikit-learn kaggle kagglehub pandas

Login using your HuggingFace token

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

## Configure Dataset

In [ ]:
# please add your kaggle.json file in the same directory as this notebook
# to access the dataset at kaggle hub

import kagglehub

path = kagglehub.dataset_download("kausthubkannan/ai-and-human-art-classification")
print("Path to dataset files:", path)

In [ ]:
from pathlib import Path

input_dir = Path('../root/.cache/kagglehub/datasets/kausthubkannan/ai-and-human-art-classification/versions/1/ai_art_classification/train')

categories = [x.name for x in input_dir.iterdir() if x.is_dir()]
categories

In [ ]:
data = []
labels = []

for category in categories:
  for file_path in (input_dir / category).iterdir():
    if file_path.is_file():
      labels.append(category)
      data.append(str(file_path))

In [ ]:
import pandas as pd

df = pd.DataFrame({'image': data, 'label': labels})
df.head()

In [ ]:
df.shape

In [ ]:
df = df.sample(frac=0.1)
df.shape

In [ ]:
df['label'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.15, stratify=df["label"], random_state=42)

In [ ]:
train_df.reset_index(inplace=True)
test_df.reset_index(inplace=True)

In [ ]:
train_df.drop(['index'], axis=1, inplace=True)
test_df.drop(['index'], axis=1, inplace=True)

In [ ]:
train_df, validation_df = train_test_split(train_df, test_size=0.20, stratify=train_df["label"], random_state=42)

In [ ]:
label_to_num = {
  'AI_GENERATED': '0',
  'NON_AI_GENERATED': '1'
}

num_to_label = {
  '0': 'AI_GENERATED',
  '1': 'NON_AI_GENERATED'
}

In [ ]:
train_df.loc[:, 'label'] = [label_to_num[label] for label in train_df['label']]
validation_df.loc[:, 'label'] = [label_to_num[label] for label in validation_df['label']]
test_df.loc[:, 'label'] = [label_to_num[label] for label in test_df['label']]

In [ ]:
train_df.reset_index(drop=True, inplace=True)
validation_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [ ]:
train_df

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict, Features, Image, Value

features = Features({
    'image': Image(),
    'label': Value(dtype='int64')
})

train_dataset = Dataset.from_pandas(train_df, features=features)
validation_dataset = Dataset.from_pandas(validation_df, features=features)
test_dataset = Dataset.from_pandas(test_df, features=features)

datasets = {
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset
}

ai_and_human_art_dataset = DatasetDict(datasets)

In [ ]:
ai_and_human_art_dataset

In [ ]:
ai_and_human_art_dataset["train"][0]

In [ ]:
from transformers import AutoImageProcessor

model_name = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(model_name)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

size = (image_processor.size["height"], image_processor.size["width"])

train_data_augmentation = keras.Sequential(
    [
        layers.RandomCrop(size[0], size[1]),
        layers.Rescaling(scale=1.0/127.5, offset=-1),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2)
    ],
    name="train_data_augmentation"
)

val_data_augmentation = keras.Sequential(
    [
        layers.CenterCrop(size[0], size[1]),
        layers.Rescaling(scale=1.0/127.5, offset=-1)
    ],
    name="val_data_augmentation"
)

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image

def convert_to_tf_tensor(image: Image):
    np_image = np.array(image)
    tf_image = tf.convert_to_tensor(np_image)
    return tf.expand_dims(tf_image, 0)

def preprocess_train(example_batch):
    images = [
        train_data_augmentation(convert_to_tf_tensor(image.convert("RGB"))) for image in example_batch["image"]
    ]
    example_batch["pixel_values"] = [tf.transpose(tf.squeeze(image)) for image in images]
    return example_batch

def preprocess_val(example_batch):
  images = [
      val_data_augmentation(convert_to_tf_tensor(image.convert("RGB"))) for image in example_batch["images"]
  ]
  example_batch["pixel_values"] = [tf.transpose(tf.squeeze(image)) for image in images]
  return example_batch

In [ ]:
ai_and_human_art_dataset["train"].set_transform(preprocess_train)
ai_and_human_art_dataset["test"].set_transform(preprocess_val)

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import create_optimizer

batch_size = 32
num_epochs = 5
num_train_steps = len(ai_and_human_art_dataset["train"]) * num_epochs
learning_rate = 3e-5
weight_decay_rate = 0.01

optimizer, lr_schedule = create_optimizer(
    init_lr=learning_rate,
    num_train_steps=num_train_steps,
    weight_decay_rate=weight_decay_rate,
    num_warmup_steps=0
)

In [ ]:
from transformers import TFAutoModelForImageClassification

model = TFAutoModelForImageClassification.from_pretrained(
    model_name,
    id2label = num_to_label,
    label2id = label_to_num
)

In [ ]:
tf_train_dataset = ai_and_human_art_dataset["train"].to_tf_dataset(
    columns="pixel_values",
    label_cols="label",
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator
)

tf_eval_dataset = ai_and_human_art_dataset["train"].to_tf_dataset(
    columns="pixel_values",
    label_cols="label",
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator
)

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

loss = SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss)

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback, PushToHubCallback

metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics,
    eval_dataset=tf_eval_dataset
)

push_to_hub_callback = PushToHubCallback(
    output_dir="ai-and-human-art-classifier",
    tokenizer=image_processor,
    save_strategy="no"
)

callbacks = [metric_callback, push_to_hub_callback]

In [ ]:
model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    epochs=num_epochs,
    callbacks=callbacks
)